In [ ]:
#https://www.bovada.lv/sports/horses-futures-props
#Also try GradientBoosted

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import math

In [40]:
df_horses = pd.read_csv("horsesNoRace.csv", header=0) 

In [ ]:
list(df_horses)

In [ ]:
df_horses.shape

In [42]:
#type(df_horses['TrackCode'])
df_horses['TrackCode']

0        658185
1        658185
2        658185
3        658185
4        658185
5        658185
6        658185
7        658185
8        658185
9        658185
10       658185
11       658185
12       658185
13       658185
14       658185
15       658185
16       658185
17       658185
18       658185
19       658185
20       658185
21       658185
22       658185
23       658185
24       658185
25       658185
26       658185
27       658185
28       658185
29       658185
          ...  
33703    907365
33704    907365
33705    907365
33706    907365
33707    907365
33708    907365
33709    907365
33710    907365
33711    907365
33712    907365
33713    907365
33714    907365
33715    907365
33716    907365
33717    907365
33718    907365
33719    907365
33720    907365
33721    907365
33722    907365
33723    907365
33724    907365
33725    907365
33726    907365
33727    907365
33728    907365
33729    907365
33730    907365
33731    907365
33732    907365
Name: TrackCode, Length:

In [41]:
# import hashlib
df_horses.dropna(subset=['TrackCode'])
def convertToNumber (s):
    stringVal = ""
    #if isinstance(s, str):
    #    print("Found not string %s %s", s, type(s))
    #print("Type and val %s %s", s, type(s))
    for c in s:
        stringVal += str(ord(c))
    return int(stringVal)
    #return ''.join(str(ord(c)) for c in s)
#Combine columns for unique id
# Merge cols 2,3,4, remove '.'s
#Need to drop last row
df_horses.drop(df_horses.index[len(df_horses)-1])
df_horses['TrackCode'] = df_horses['TrackCode'].apply(lambda x: convertToNumber(x))

In [43]:
#Add header?


df_horses['raceID'] = df_horses['TrackCode'].map(str) + df_horses['Date'].map(str) + df_horses['RaceNumber'].map(str)
df_horses['raceID'] = df_horses['raceID'].replace(to_replace='\.', value='', regex=True)

df_horses.head()

,RecordType,TrackCode,Date,RaceNumber,Day,LastRace,BreedType,HorseName,HorseWeight,HorseAge,...,Owner,Comments,WinnersInfo,payWin,payPlace,payShow,payShowedHeat,extra1,extra2,raceID
0,H,658185,20181114,1,D,26Oct18 9BEL12,TB,Won't Be Missed,110,3,...,T L C Racing,"WON'T BE MISSED forwardly placed 3w, coaxed 7/...",NaN,0.0,0,0.0,0.0,NaN,NaN,658185201811141
1,H,658185,20181114,1,D,22Sep18 12PRX 6,TB,Purchasing Power,120,3,...,"Klaravich Stables, Inc.","PURCHASING POWER broke in st, at rear 5-6w, co...",NaN,0.0,2.2,2.1,0.0,NaN,NaN,658185201811141
2,H,658185,20181114,1,D,14Oct18 6BEL 6,TB,Malibu Strings,118,3,...,Kantarmaci Racing LLC,"MALIBU STRINGS unhurried 4-5w, coaxed 4w 3/8, ...",Bay Colt,2015.0,Malibu Moon-Meteor Miracle (Twining) Bred By:...,11.8,3.8,2.1,0.0,658185201811141
3,H,658185,20181114,1,D,13Oct18 3BEL 1,TB,Magnetron,120,3,...,South of Fifth Racing,"MAGNETRON 3w 1st turn, 2-3w early, to ins 1/2p...",NaN,0.0,0,2.1,0.0,NaN,NaN,658185201811141
4,H,658185,20181114,1,D,30Sep18 4PRX 6,TB,Incubator,118,3,...,George E Hall,"INCUBATOR away well, spurted in hand 2p, set d...",NaN,0.0,0,0.0,0.0,NaN,NaN,658185201811141


In [44]:
# Data analysis
df_horses['raceID']

0         658185201811141
1         658185201811141
2         658185201811141
3         658185201811141
4         658185201811141
5         658185201811142
6         658185201811142
7         658185201811142
8         658185201811142
9         658185201811142
10        658185201811142
11        658185201811142
12        658185201811143
13        658185201811143
14        658185201811143
15        658185201811143
16        658185201811143
17        658185201811143
18        658185201811143
19        658185201811143
20        658185201811143
21        658185201811143
22        658185201811143
23        658185201811143
24        658185201811143
25        658185201811143
26        658185201811143
27        658185201811143
28        658185201811144
29        658185201811144
               ...       
33703     907365201812098
33704     907365201812098
33705     907365201812098
33706     907365201812098
33707     907365201812098
33708     907365201812098
33709     907365201812098
33710     90

In [45]:
# Check for best odds
# Remove where odds are 0
df_FixedOdds = df_horses.dropna(subset=['JockeysName'])   


In [46]:
#df_horses.groupby('raceID')['Odds'].idxmin(skipna=True)
df_Min = df_FixedOdds.loc[df_FixedOdds.groupby('raceID')['Odds'].idxmin()]

In [47]:
#Double check this, most of the odds are 0
#Odds = 0 race is scratched
wins = 0
loss = 0
money = 0
currentLosingStreak = 0
maxLosingStreak = 0
numBets = 0


for index, row in df_Min.iterrows():
    numBets += 1
    if row['Odds'] == 0:
        print("Empty odds found")
    elif row['OfficialFinish'] == 1:
        wins += 1
        money = money + row['Odds']
        currentLosingStreak = 0
    else:
        loss += 1
        money = money - 1
        currentLosingStreak = currentLosingStreak + 1
        if currentLosingStreak > maxLosingStreak:
            maxLosingStreak = currentLosingStreak
            
print ("NumBets: {}".format(numBets))
print ("Wins: {}".format(wins))
print ("Loss: {}".format(loss))
print ("Money: {}".format(money))
print ("Max Losing Streak: {}".format(maxLosingStreak))

#Expect ratio to be 1/2


Empty odds found
Empty odds found
Empty odds found
Empty odds found
Empty odds found
Empty odds found
Empty odds found
NumBets: 3669
Wins: 1338
Loss: 2324
Money: -710.1000000000001
Max Losing Streak: 17


In [ ]:
#For modeling remove the finish place and prize money, along with other additional features that show the end of the race

In [48]:
#RaceNumber, Odds, HorseWeight, HorseAge, PostPos
df_features = df_FixedOdds[['RaceNumber', 'Odds', 'HorseWeight', 'HorseAge', 'PostPos', 'raceID']]

In [49]:
#Cat Features, TrackCode, JockeyName, HorseName, BreedType, Sex, Owner
categorical_features = ['TrackCode', 'JockeysName', 'HorseName', 'BreedType', 'Sex', 'Owner']

for feature in categorical_features:
    df_FixedOdds[feature] = df_FixedOdds[feature].astype(str)
    df_FixedOdds[feature] = df_FixedOdds[feature].replace('nan','0')
    
for feature in categorical_features:
    encoded_features = pd.get_dummies(df_FixedOdds[feature])
    encoded_features.columns = feature + encoded_features.columns
    df_features = pd.merge(df_features,encoded_features,left_index=True,right_index=True,how='inner')

#Handle target var
df_features['win'] = False
df_features.loc[df_FixedOdds['OfficialFinish'] == 1,'win'] = True   
    

c:\users\jeff\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\jeff\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
#TODO: Engineer Last Race
from sklearn.model_selection import train_test_split
# Parameters are features, target, split ratio, random seed
X_train, X_test, y_train, y_test = train_test_split(df_features.loc[:, df_features.columns != 'win'], df_features['win'] ,test_size=0.20, random_state=42)

In [51]:
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 23980 samples.
Testing set has 5996 samples.


In [52]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0)


In [53]:
from sklearn.metrics import fbeta_score, accuracy_score
from time import time
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end-start
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    #print (type(predictions_test))
    #print (predictions_train)
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
    
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train)
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = fbeta_score(y_train[:300], predictions_train, 0.5)
        
    # TODO: Compute F-score on the test set which is y_test
    # 3rd param is Beta
    results['f_test'] = fbeta_score(y_test, predictions_test, 0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results


In [54]:
#print(y_train.size)
samples_100 = y_train.size
train_predict(clf, samples_100, X_train, y_train, X_test, y_test)

RandomForestClassifier trained on 23980 samples.


c:\users\jeff\appdata\local\programs\python\python35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'acc_test': 0.8780853902601734,
 'acc_train': 0.84666666666666668,
 'f_test': 0.0,
 'f_train': 0.0,
 'pred_time': 14.13710618019104,
 'train_time': 107.97836351394653}

In [55]:
predictions = clf.predict_proba(X_test)[:,0]

In [56]:
X_test.columns

Index(['RaceNumber', 'Odds', 'HorseWeight', 'HorseAge', 'PostPos', 'raceID',
       'TrackCode658185', 'TrackCode676832', 'TrackCode677782',
       'TrackCode678432',
       ...
       'OwnerZolotas, Steven and Zolotas, Sabina Romo', 'OwnerZonie Chavers',
       'OwnerZoom and Fish Stable Inc and Spiring Charlie',
       'OwnerZoulias Nick Chin Sherman and Allinson Patrick J',
       'Ownerde Tomaso Isabelle and Jones Hope',
       'Ownere Five Racing Thoroughbreds',
       'Ownere Five Racing Thoroughbreds Brennan Niall J and Ryan Michael J',
       'Ownere Five Racing Thoroughbreds and Magic Cap Stables',
       'Ownere Five Racing Thoroughbreds and Stonestreet Stables LLC',
       'Ownerkaylafarms LLC'],
      dtype='object', length=32780)

In [57]:
predictions = clf.predict_proba(X_test)[:,0]

In [58]:
X_test['predictions'] = predictions

In [75]:
#Win is in y_test


df_test = X_test[['predictions','raceID']]

In [76]:
df_test['win'] = y_test

c:\users\jeff\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [81]:
df_test['Odds'] = X_test['Odds']

c:\users\jeff\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [82]:
df_profit = df_test.loc[df_test.groupby("raceID")["predictions"].idxmax()]
df_profit
investment = 0
payout = 0
for index, row in df_profit.iterrows():
    investment +=1
    
    if (row['win']):
        payout += row['Odds']

investment_return = round((payout - investment)/investment*100,2)
print("This algorithm and betting system will generate a " + str(investment_return) + "% return\n")
print("Note: you can't read much from a single run. Best to setup a cross validation framework and look at the return over many runs")


This algorithm and betting system will generate a -40.87% return

Note: you can't read much from a single run. Best to setup a cross validation framework and look at the return over many runs


In [83]:
df_test

,predictions,raceID,win,Odds
25453,0.883519,806978201812266,False,8.10
27452,0.883282,828032201812059,False,96.10
22448,0.884366,778682201811194,False,9.40
15577,0.883282,718087201811172,False,7.80
17807,0.869192,678432201811282,False,3.80
30849,0.872563,848032201812077,False,4.30
1892,0.884879,677782201811297,False,52.55
24172,0.879090,778682201812266,True,4.70
26997,0.884366,828032201811241,False,7.20
28776,0.882386,846577201812088,False,4.80


In [ ]:
clf.importances